In [1]:
import redis

In [2]:
redis_conn = redis.Redis(host='localhost', port=6379, db=0)

In [ ]:
for key in redis_conn.keys():
    redis_conn.delete(key)

In [5]:
redis_conn.set('foo1', 'bar')
redis_conn.get('foo')

ConnectionError: Error 61 connecting to localhost:6379. Connection refused.

In [23]:
# 使用hash，类map的形式，存储（id，标题）数据
for idx in range(1, 10):
    redis_conn.hset('article', idx, 'title' + str(idx))

In [ ]:
# 展示所有的文章列表
for article_id, article_title in redis_conn.hgetall('article').items():
    print("#"*30)
    print(article_id, article_title)
    print(article_id.decode("utf-8"), article_title.decode("utf-8"))

In [28]:
# 展示单个文章的标题  key id
redis_conn.hget('article', '1')

b'title1'

In [29]:
def user_visit(uid, article_id):
    # string
    redis_conn.incr(f"article_counter_{article_id}")
    # List
    redis_conn.lpush(f"user_visit_{uid}", article_id)
    # Set
    redis_conn.sadd(f"all_visit_uids", str(uid))
    # SortedSet
    redis_conn.zincrby("article_hots", 1, article_id)

In [30]:
user_visit("uid_01","1")
user_visit("uid_01","2")
user_visit("uid_01","3")

user_visit("uid_02","2")
user_visit("uid_02","3")
user_visit("uid_02","4")

user_visit("uid_03","4")
user_visit("uid_03","5")
user_visit("uid_03","6")

In [37]:
redis_conn.get(f"article_counter_1")

b'2'

In [38]:
redis_conn.lrange("user_visit_uid_01", 0, -1)

[b'3', b'2', b'1']

In [39]:
redis_conn.lrange("user_visit_uid_02", 0, -1)

[b'4', b'3', b'2']

In [40]:
redis_conn.smembers("all_visit_uids")

{b'uid_01', b'uid_02', b'uid_03'}

In [44]:
redis_conn.zrange("article_hots", 0, -1, withscores=True, desc=True)

[(b'4', 2.0), (b'3', 2.0), (b'2', 2.0), (b'6', 1.0), (b'5', 1.0), (b'1', 1.0)]